<a href="https://colab.research.google.com/github/GabrielFCosta/TCC-BSI-2023.1/blob/main/old/matc97_gabriel_teste_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import string
import nltk
import math
from urllib.request import urlopen
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# Funções do reconhecimento de entidades nomeadas:

# Remove pontuação, utilizada pela função seguinte de pré-processamento
def remover_pontuacao(tokens):
  for token in tokens:
    if token in string.punctuation:
      tokens.remove(token)
# Função de pré-processamento
def preprocess(text):
    text = word_tokenize(text)
    remover_pontuacao(text)
    return pos_tag(text)

# Listas para coletar labels e entidades pra cada sentença lida do texto
codes = []
words = []
def  clearlists():
  if len(words) > 0:
    words.clear()
  if len(codes) > 0:
    codes.clear()

# Função de reconhecimento de entidades nomeadas
def ner(sent):
  ne_tree = nltk.ne_chunk(sent)
  for chunk in ne_tree:
    if hasattr(chunk,"label"):
      word =' '.join(c[0] for c in chunk)
     #print(chunk.label(),word)
      codes.append(chunk.label())
      words.append(word)

# Funções para criar um dataframe contendo entidades, labels e suas frequencias 
def returndataset():
  dataset = datajoin(words,codes)
  # calcula frequências das entidades e adiciona nova coluna
  dataset['freq'] = dataset.groupby(['entity','label'])['entity'].transform('count')
  return dataclean(dataset)
def datajoin(words,codes):
  # junta as duas listas de entidades e labels e um dataframe
  dataset = pd.DataFrame(zip(words,codes),columns =['entity', 'label'])
  clearlists()
  return dataset
def dataclean(dataset):
  # remove linhas duplicatas e atualiza index
  dataset.drop_duplicates(inplace= True)
  dataset.reset_index()
  return dataset

# Calcula TF (Term frequency) das entidades e adiciona coluna ao dataframe
def calculaTF(dataset):
  tfs = []
  # frequência do termo mais frequente no documento 
  max = dataset['freq'].max()
  # calcula TF para cada entidade no documento
  for value in dataset['freq']:
    tfs.append(value/max)
  dataset['TF'] = tfs
  return dataset

# Calcula IDF (Inverse Document Frequency) e retorna dataframe
def retornaIDF(datasets,c):
  clearlists()
  # coleta entidades do primeiro documento
  for index,row in datasets[0].iterrows():
     words.append(row['entity'])
     codes.append(row['label'])
  # coleta entidades do documento a ser comparado
  for index, row in datasets[c].iterrows():
      words.append(row['entity'])
      codes.append(row['label'])
  # Produz dataframe de todas as entidades e respectivas frequências no corpus
  idfs = returndataset()
  # total de documentos no corpus será sempre 2 (dois documentos sendo comparados)
  N = 2
  aux = []
  # Calcula IDF (utilizando ln) para cada entidade e adiciona coluna ao dataframe
  for index, row in idfs.iterrows():
    aux.append(math.log(N/row['freq']))
  idfs['IDF'] = aux
  return idfs 

# Retorna dataframe com pesos para comparação entre documentos
def calculaPeso(datasets,indexdoc,indexidf):
 idf = retornaIDF(datasets,indexidf)
 # produz um dataframe composto pela união das TFs e IDFs de todas entidades dos dois documentos.  
 #wij = pd.merge(datasets[indexdoc].loc[:,datasets[indexdoc].columns!="freq"], idf.loc[:,idf.columns!="freq"], on=['entity','label'], how='outer')
 wij = pd.merge(datasets[indexdoc], idf.loc[:,idf.columns!="freq"], on=['entity','label'], how='outer')
 # preenche valores de IDF NaN com 0
 wij = dataclean(wij).fillna (0)
 # cria outra coluna com o valores dos pesos: coluna IDF * coluna TF
 wij["Wij"] = wij['IDF'] * wij['TF']
 # retorna dataframe ordenado
 return wij.sort_values(by=['entity','label'])

# similaridade de jaccard simplificada 
def similaridade_jaccard(freqA, freqN):
    intersecao = 0
    c = 0
    for x in freqA:
      if x == freqN[c]:
        intersecao += 1
      c += 1
    print("interseção:",intersecao)
    uniao = len(freqA)
    print("união:",uniao)
    print("jaccard simplificada:",intersecao/uniao)

# fórmula da similaridade de jaccard utilizando frequências absolutas
def jaccard(freqA, freqN):
  numerador = returnumerador(freqA, freqN)
  denominador = 0 
  for valor in freqA:
    denominador += valor * valor
  for valor in freqN:
    denominador += valor * valor
  denominador = denominador - numerador
  print("jaccard fórmula:", numerador/denominador)

def returnumerador(freqA, freqN):
  numerador = 0
  c = 0
  for valor in freqA:
    numerador += valor * freqN[c]
    c+=1
  return numerador
  
# (false)
def cosine(freqA, freqN):
  numerador = returnumerador(freqA, freqN)
  den0 = 0
  den1 = 1
  for val0 in freqA:
    den0 += val0 * val0
  for val1 in freqN:
    den1 += val1 * val1
  denominador = math.sqrt(den0) * math.sqrt(den1)
  print("cosine:", numerador/denominador)



In [ ]:
# Execução do reconhecimento de entidades nomeadadas:

#urls = ["https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/noticia.txt",
#        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/noticia1.txt",
#        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/noticia2.txt"]
# Lista que vai conter dataframes de entidades (corpus)
datasets = []

urls = ["Airbnb will shut down its listings in China after two years of lockdowns in the country",
        "Starting this summer, Airbnb will take down its listings and offers for hosted experiences in China.",
        "International brands, from Apple to Estee Lauder, have warned of the financial impact of the restrictions."]
# Pra cada url
for url in urls:
  #data = urlopen(url)
  #ex = data.read().decode('utf-8')
  ex=nltk.sent_tokenize(url)
  print(ex)
  # Detectando entidades de nomeadas por sentença
  for sentence in ex:
    # Sentença Pré-processada
    text = preprocess(sentence)
    print(text)
    # Entidades Nomeadas na Sentença
    ner(text)
  # Adiciona dataframe de entidades nomeadas do texto + labels, frequências e TF
  datasets.append(calculaTF(returndataset()))

c = 0
for dataset in datasets:
  print("\nDocumento:",c)
  print(dataset)
  c+=1
for x in range(c-1):
  print("\nTabela IDF para documentos 0 e",x+1)
  print(retornaIDF(datasets,x+1))


['Airbnb will shut down its listings in China after two years of lockdowns in the country']
[('Airbnb', 'NNP'), ('will', 'MD'), ('shut', 'VB'), ('down', 'RP'), ('its', 'PRP$'), ('listings', 'NNS'), ('in', 'IN'), ('China', 'NNP'), ('after', 'IN'), ('two', 'CD'), ('years', 'NNS'), ('of', 'IN'), ('lockdowns', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('country', 'NN')]
['Starting this summer, Airbnb will take down its listings and offers for hosted experiences in China.']
[('Starting', 'VBG'), ('this', 'DT'), ('summer', 'NN'), ('Airbnb', 'NNP'), ('will', 'MD'), ('take', 'VB'), ('down', 'RP'), ('its', 'PRP$'), ('listings', 'NNS'), ('and', 'CC'), ('offers', 'NNS'), ('for', 'IN'), ('hosted', 'JJ'), ('experiences', 'NNS'), ('in', 'IN'), ('China', 'NNP')]
['International brands, from Apple to Estee Lauder, have warned of the financial impact of the restrictions.']
[('International', 'NNP'), ('brands', 'NNS'), ('from', 'IN'), ('Apple', 'NNP'), ('to', 'TO'), ('Estee', 'NNP'), ('Lauder', 'NNP'), ('hav

In [ ]:
print("\nComparando documento 0 consigo mesmo (caso base):")
print("\nTabela de pesos para documentos 0")
wij0 = calculaPeso(datasets,0,0)
print(wij0)
similaridade_jaccard(wij0['freq'].tolist(), wij0['freq'].tolist())
jaccard(wij0['freq'].tolist(), wij0['freq'].tolist())
#cosine(wij0['freq'].tolist(), wij0['freq'].tolist())

print("\nComparando documentos 0 e 1:")
print("\nTabela de pesos para documentos 0")
wij0 = calculaPeso(datasets,0,1)
print(wij0)
print("\nTabela de pesos para documentos 1")
wij1 = calculaPeso(datasets,1,1)
print(wij1)
print("\nSimilaridade entre documentos 0 e 1:")
similaridade_jaccard(wij0['freq'].tolist(), wij1['freq'].tolist())
jaccard(wij0['freq'].tolist(), wij1['freq'].tolist())
#cosine(wij0['freq'].tolist(), wij1['freq'].tolist())

print("\nComparando documentos 0 e 2:")
print("\nTabela de pesos para documentos 0")
wij0 = calculaPeso(datasets,0,2)
print(wij0)
print("\nTabela de pesos para documentos 1")
wij1 = calculaPeso(datasets,2,2)
print(wij1)
print("\nSimilaridade de jaccard entre documentos 0 e 2:")
similaridade_jaccard(wij0['freq'].tolist(), wij1['freq'].tolist())
jaccard(wij0['freq'].tolist(), wij1['freq'].tolist())
#cosine(wij0['freq'].tolist(), wij1['freq'].tolist())


Comparando documento 0 consigo mesmo (caso base):

Tabela de pesos para documentos 0
   entity label  freq   TF  IDF  Wij
0  Airbnb   GPE     1  1.0  0.0  0.0
1   China   GPE     1  1.0  0.0  0.0
interseção: 2
união: 2
jaccard simplificada: 1.0
jaccard fórmula: 1.0

Comparando documentos 0 e 1:

Tabela de pesos para documentos 0
   entity   label  freq   TF       IDF       Wij
0  Airbnb     GPE   1.0  1.0  0.693147  0.693147
2  Airbnb  PERSON   0.0  0.0  0.693147  0.000000
1   China     GPE   1.0  1.0  0.000000  0.000000

Tabela de pesos para documentos 1
   entity   label  freq   TF       IDF       Wij
2  Airbnb     GPE   0.0  0.0  0.693147  0.000000
0  Airbnb  PERSON   1.0  1.0  0.693147  0.693147
1   China     GPE   1.0  1.0  0.000000  0.000000

Similaridade entre documentos 0 e 1:
interseção: 1
união: 3
jaccard simplificada: 0.3333333333333333
jaccard fórmula: 0.3333333333333333

Comparando documentos 0 e 2:

Tabela de pesos para documentos 0
         entity   label  freq   TF    

In [ ]:

# Funções da recomendação por conteúdo:

# Carregando o conjunto de dados
data_set = pd.read_csv("https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/sample-data.csv")
print(data_set)
# Utilizando o TfId para vetorizar o name dos items
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

# Atributo utilizado: país de origem
tfidf_matrix = tf.fit_transform(data_set['countryOfOrigin'])

# Calculando a matriz de similaridade cosseno
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_similarities)

# Apresenta itens com mesmo país de origem
resultados= []
i=0
for sims in cosine_similarities[0]:
    if(sims>0):
        resultados.append(data_set.loc[i,'name'])
    i+=1
print("\n Recomendando produtos similares a " + resultados[0] + ",com base no país de origem")
for x in range(1,len(resultados)):
    print(resultados[x])

   id                 genericName                                        name  \
0   1                      Butter                  Land O’Lakes Salted Butter   
1   2              Cheddar cheese                 Kraft Cheddar Cheese Slices   
2   3              Vanilla Yogurt  Yoplait Light ‘n Fit Nonfat Vanilla Yogurt   
3   4  Chocolate Yogurt Ice Cream         Breyer’s Chocolate Yogurt Ice Cream   
4   5                      Butter                        Banana Salted Butter   
5   6                        milk                  Land O’Lakes full fat milk   

  countryOfOrigin  
0           China  
1              UK  
2          France  
3           china  
4             USA  
5         chinese  
[[1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

 Recomendando produtos similares a Land O’Lakes Salted Butter,com base no país de origem
Breyer’s Chocolate Yogurt Ice Cream


In [ ]:

def test(tag):
  # todos substantivos, adjetivos, verbos e numerais.
  tags = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ','CD']
  for i in tags:
    if i == tag:
      return True
  return False

def filtertuples(text):
  words = []
  tags = []
  for tuple in text:
    if test(tuple[1]):
      words.append(tuple[0])
      tags.append(tuple[1])
  palavras = pd.DataFrame(zip(words,tags),columns =['word', 'pos_tag'])
  palavras['freq'] = palavras.groupby(['word','pos_tag'])['word'].transform('count')
  #palavras = calculaTF(palavras)
  palavras = dataclean(palavras)
  return palavras

def datamerge(words0,words1):
  merged = pd.merge(words0,words1, on=['word','pos_tag'], how='outer')
  merged = dataclean(merged).fillna(0)
  doc_freq = []
  c = 0
  for i in merged['freq_x'].tolist():
    if i > 0 and merged['freq_y'].tolist()[c] > 0:
      doc_freq.append(2)
    else:
      doc_freq.append(1)
    c += 1
  merged['doc_freq'] = doc_freq
  return merged.sort_values(by=['word','pos_tag'])

def jaccard_simples(doc_freq):
    intersecao = 0
    c = 0
    for x in doc_freq:
      if x == 2:
        intersecao += 1
      c += 1
    print("interseção:",intersecao)
    uniao = len(doc_freq)
    print("união:",uniao)
    print("jaccard simplificada:",intersecao/uniao)

def similaridade_WP_means(str0,str1):
  means = []
  for wd0 in str0:
    syn0 = wn.synsets(wd0)
    # verifica se synset0 contém alguma definição
    if len(syn0) > 0:
      val = 0
      for wd1 in str1:
        syn1 = wn.synsets(wd1)
        # verifica se synset1 contém alguma definição
        if len(syn1) > 0:
          val+= syn0[0].wup_similarity(syn1[0])
          #print("syn0:",syn0[0],"syn1:",syn1[0],"val:",val)
      means.append(val/len(str1))
  print(means)
  print(sum(means)/len(means))

def similaridade_WP_max(str0,str1):
  means = []
  vals = []
  aux = 0
  for wd0 in str0:
    syn0 = wn.synsets(wd0)
    # verifica se 1a palavra contém alguma definição antes de prosseguir
    if len(syn0) != 0:
      vals.clear()
      for wd1 in str1:
        syn1 = wn.synsets(wd1)
        # verifica se 2a palavra contém alguma definição antes de calcular similaridade
        if len(syn1) != 0:
          aux = syn0[0].wup_similarity(syn1[0])
          # se similaridade = 1, palavras iguais, break
          if aux == 1:
            break
          # senão, adiciona valor à lista vals
          else:
            vals.append(aux)
            #print("syn0:",syn0[0],"syn1:",syn1[0],"wup:",syn0[0].wup_similarity(syn1[0]))
      # se aux = 1, similaridade máxima = 1
      if aux == 1:
         means.append(aux)
      # senão seleciona máxima da lista
      else:
        means.append(max(vals))
  print(means)
  print(sum(means)/len(means))

str0 = filtertuples(preprocess(urls[0]))
str1 = filtertuples(preprocess(urls[1]))
str2 = filtertuples(preprocess(urls[2]))

print("string 0:",str0['word'].tolist())
print("string 1:",str1['word'].tolist())
similaridade_WP_means(str0['word'].tolist(),str1['word'].tolist())
similaridade_WP_max(str0['word'].tolist(),str1['word'].tolist())
dt = datamerge(str0,str1)
jaccard_simples(dt['doc_freq'].tolist())

print('\n')
print("string 0:",str0['word'].tolist())
print("string 2:",str2['word'].tolist())
similaridade_WP_means(str0['word'].tolist(),str2['word'].tolist())
similaridade_WP_max(str0['word'].tolist(),str2['word'].tolist())
dt = datamerge(str0,str2)
jaccard_simples(dt['doc_freq'].tolist())

# cosine do scikit
print('\ncosine do scikit')
corpus = [urls[0],urls[1],urls[2]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
cosine_similarity(trsfm[0:1], trsfm)

string 0: ['Airbnb', 'shut', 'listings', 'China', 'two', 'years', 'lockdowns', 'country']
string 1: ['Starting', 'summer', 'Airbnb', 'take', 'listings', 'offers', 'hosted', 'experiences', 'China']
[0.17105672105672104, 0.29106753812636166, 0.1999273783587509, 0.21296296296296294, 0.25932150638032986, 0.26241529750301684, 0.19814814814814813]
0.22784279321947018
[0.25, 1.0, 1.0, 0.4, 0.7142857142857143, 0.631578947368421, 0.26666666666666666]
0.6089330469029717
interseção: 3
união: 14
jaccard simplificada: 0.21428571428571427


string 0: ['Airbnb', 'shut', 'listings', 'China', 'two', 'years', 'lockdowns', 'country']
string 2: ['International', 'brands', 'Apple', 'Estee', 'Lauder', 'have', 'warned', 'financial', 'impact', 'restrictions']
[0.20388888888888887, 0.18476190476190474, 0.16702548225768038, 0.172291923762512, 0.18476190476190474, 0.17561612364243942, 0.18405662964486494]
0.1817718368171707
[0.5, 0.2857142857142857, 0.35294117647058826, 0.26666666666666666, 0.2857142857142857, 0

array([[1.        , 0.35608377, 0.15772405]])